In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sp
from scipy.sparse import csr_matrix
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from collections import Counter
import pickle

In [2]:
user_df = pd.read_excel('data/final_user_dataframe_transposed.xlsx', index_col=0)

In [3]:
user_df.shape
user_df = user_df.replace(0, np.nan) #replacing all 0 values with NaN

num_of_users = len(user_df)
num_of_beers = len(user_df.columns)
print(f'Number of Users: {num_of_users}')
print(f'Number of Beers: {num_of_beers}')

Number of Users: 101
Number of Beers: 629


In [4]:
user_df = user_df.dropna(axis=1, how='all') #Dropping all beers that do not have ratings
user_df.shape

(101, 338)

In [6]:
avg_per_beer = np.mean(user_df) #Calculating averages
top_rated = avg_per_beer.sort_values(ascending=False).head(10)

user_averages = user_df.mean(axis=1) #Calculating average rating by user
user_averages

DEE               4.697987
DON               5.523810
SAM               6.396947
EMILIA            4.750000
MEAGAN            6.720280
                    ...   
Ray               6.636364
Mike              6.736842
Leo               5.500000
Matt McInerney    6.625000
Jenny             6.000000
Length: 101, dtype: float64

In [7]:
#Because not all beers have the same number of ratings, its standard to calculate the Bayesian averages.
beer_stats = user_df.agg(['count', 'mean'])

beer_stats = beer_stats.T
beer_stats = beer_stats.reset_index()
beer_stats.columns =['beer', 'count', 'average']

C = beer_stats['count'].mean()
m = beer_stats['average'].mean()

def bayesian_avg(rating):
    bayesian_avg = (C*m+rating.sum())/(C+rating.count())
    return bayesian_avg

In [8]:
bayesian_avg_ratings = user_df.agg(bayesian_avg).reset_index()
bayesian_avg_ratings.columns = ['beer', 'bayesian_avg']
beer_stats = beer_stats.merge(bayesian_avg_ratings, on='beer')
beer_stats = beer_stats.sort_values(by='bayesian_avg', ascending=False)

top_five_beers = beer_stats.head(5)
top_five_beers

,beer,count,average,bayesian_avg
220,High Road Brewing: Bronan,13.0,8.846154,7.834045
19,Bellwoods Brewery: Jelly King,74.0,7.797297,7.640132
34,High Road Brewing: Cloud Piercer,12.0,8.500000,7.574132
77,Bellwoods Brewery: Jelly King - Pink Guava,12.0,8.333333,7.473507
29,Brewery Ommegang: Ommegang Pale Sour,14.0,7.785714,7.201653


These beers have the highest Bayesian average. Although this is clearly a bias from the fact that 12 of the 15 people who rated the entire beer list work at Trinity Common. I've only ever seen Cloud Piercer on tap at TC, and Bronan is very rare in Toronto (and exists at TC as a permanent tap).


In [9]:
lowest_ranking_beers = beer_stats.tail(5)
lowest_ranking_beers
#Probably a bias of the unusually high hipster quotient.

,beer,count,average,bayesian_avg
4,Heineken: Heineken Lager,99.0,4.202020,4.346557
308,Sleeman Breweries Ltd.: Honey Brown Lager,10.0,2.900000,4.337805
7,Alexander Keiths: Red Amber Ale,86.0,3.930233,4.117587
283,Molson Breweries of Canada: Canadian Light,6.0,1.000000,3.930708
2,Coors Brewing Company: Coors Light,95.0,2.968421,3.213017


In [10]:
user_df = user_df.drop(columns=['Clifford Brewing Co.: Clifford Porter.1']) ##dropping duplicate
user_df = user_df.reset_index().rename(columns={'index': 'user'})
user_df.shape

(101, 338)

In [ ]:
user_df.to_csv('processed_dataframe_not_melt.csv') #Saving a copy 

In [11]:
#Melting
new_df = pd.melt(user_df, id_vars='user', var_name='beer', value_name='rating')

new_df = new_df.sort_values('user')

new_df = new_df.dropna()

new_df.shape
new_df.head(2)

,user,beer,rating
29806,ANDREW,Nickel Brook Brewing Company: Head Stock,7.0
28897,ANDREW,Muskoka Brewery: Muskoka Harvest Ale,5.0


In [ ]:
new_df.to_csv('processed_dataframe.csv') #Saving a copy of melted format

In [12]:
new_df.head(2)

,user,beer,rating
29806,ANDREW,Nickel Brook Brewing Company: Head Stock,7.0
28897,ANDREW,Muskoka Brewery: Muskoka Harvest Ale,5.0


In [13]:
user_df.head(2)

,user,Beau's All Natural Brewing Company: Beau's Lugtread,Grupo Modelo: Corona,Coors Brewing Company: Coors Light,Sawdust City Brewing Co.: Little Norway,Heineken: Heineken Lager,Creemore Spring Brewery: Creemore Springs Premium Lager,Mill Street Brewery: 100th Meridian,Alexander Keiths: Red Amber Ale,Clifford Brewing Co.: Clifford Porter,...,Unibroue: Don De Dieu,Unibroue: Blonde de Chambly,Unibroue: À Tout Le Monde,Unibroue: A Tout Le Monde - Megadeth,Wellington Brewery: Helles Bock,Wellington Brewery: Iron Duke,Wellington Brewery: Bewitched Belgian,À la Fût: Hopfenweisse,À la Fût: Cuvée Houblonée I,À la Fût: À La Belge Triple
0,DEE,2.0,1.0,1.0,6.0,1.0,1.0,1.0,1.0,NaN,...,7.0,7.0,8.0,8.0,8.0,7.0,8.0,NaN,NaN,NaN
1,DON,4.0,5.0,3.0,5.0,2.0,NaN,4.0,1.0,6.0,...,NaN,NaN,NaN,6.0,NaN,NaN,NaN,NaN,NaN,7.0


In [14]:
#Prepping a styles df
beers = pd.read_csv('data/processed_dataframe_not_melt.csv', index_col=0)

beers = beers.drop(columns='user')

beers = beers.T
beers = beers.reset_index()

beers = beers.rename(columns = {'index': 'beer'})

beers = beers[['beer', 0]]

print(beers.shape)

beers.head(2)

(337, 2)


,beer,0
0,Beau's All Natural Brewing Company: Beau's Lug...,2.0
1,Grupo Modelo: Corona,1.0


In [15]:
og_beer = pd.read_csv('data/final_beer_df.csv', index_col=0)
og_beer['beer'] = og_beer['brewery'].map(str) + ": " + og_beer['name']
og_beer = og_beer[['beer', 'style_name', 'abv']]
og_beer.head(2)

,beer,style_name,abv
0,Beau's All Natural Brewing Company: Beau's Lug...,Kolsch,5
1,Grupo Modelo: Corona,American Light Lager,4.5


In [16]:
og_beer['abv'] = pd.to_numeric(og_beer.abv, errors='coerce') #Fixing ABV incase I want to use it in the future
len(og_beer['style_name'].unique())

158

In [17]:
print(og_beer.shape)
print(og_beer.style_name.unique())
og_beer.head(2)

(630, 3)
['Kolsch' 'American Light Lager' 'American Premium Lager'
 'American Imperial Stout' 'Flanders Red' 'Belgian Strong Ale'
 'Hefeweisen' 'APA' 'Pale Lager' 'Oatmean Stout' 'Imperial Stout'
 'West Coast IPA' 'Hazy Pale Ale' 'New England IPA' 'Stout' 'Beligan Wit'
 'American Lager' 'Amber' 'Lager' 'Pilsener' 'Pale Sour' 'Oatmeal Stout'
 'Pale Ale' 'IPA' 'New Zealand Pale Ale' 'Scottish-Style Light Ale'
 'American-Style Barley Wine Ale' 'American-Style Lager'
 'American-Style Stout' 'German-Style Pilsener'
 ' Company American Imperial Stout ' ' Company American Imperial IPA '
 ' Company Russian Imperial Stout ' 'French & Belgian-Style Saison'
 'American-Style India Pale Ale' 'English-Style Brown Ale'
 'Golden or Blonde Ale' 'Fresh "Wet" Hop Ale' 'Fruit Beer'
 'American-Style Amber/Red Ale' 'Open Category Mead'
 ' Company American Black Ale ' ' Company Baltic Porter '
 'Rye Ale or Lager with or without Yeast' 'Scottish-Style Heavy Ale'
 'Strong Ale' 'German-Style Altbier' 'Contempor

,beer,style_name,abv
0,Beau's All Natural Brewing Company: Beau's Lug...,Kolsch,5.0
1,Grupo Modelo: Corona,American Light Lager,4.5


In [18]:
#Fixing a wrong spelling
og_beer['beer'][14]
og_beer['beer'][14] = 'Sawdust City Brewing Co.: Gateway Kolsch'

/Users/kristigourlay/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [19]:
beer_df = og_beer.merge(beers, on='beer') #merging the two dfs to create a 'styles df'.
beer_df = beer_df.drop(columns=['abv', 0,])
beer_df.head(2)

,beer,style_name
0,Beau's All Natural Brewing Company: Beau's Lug...,Kolsch
1,Grupo Modelo: Corona,American Light Lager


In [21]:
style_names = beer_df['style_name'].tolist()

style_names = [x.lower() for x in style_names]
beer_df['style_name'] = style_names
beer_df.head(2)

,beer,style_name
0,Beau's All Natural Brewing Company: Beau's Lug...,kolsch
1,Grupo Modelo: Corona,american light lager


In [53]:
beer_dict = dict({'kolsch|kölsch': 'kolsch', 
                  'wheat|wit|weiss|hefeweisen': 'wheat',
                 'pilsner|pilsener': 'pilsner',
                  'cider': 'cider',
                  'imperial stout': 'imperial st',
                  'stout|american-style stout|milk stout|cream stout|oatmeal|oatmean|dry stout': 'stout',
                  'lager': 'lager',
                  'sour|flanders|wild|gose': 'sour',
                 'ipa|hazy|hop': 'ipa',
                  'fruit': 'fruit beer',
                  'belgian': 'belgian',
                  'mild|red|amber': 'red, amber',
                  'pale|apa': 'pale ale',
                  'bitter|brown|bock|porter': 'browns',
                  'black': 'black ale',
                  'scotch|rye|barrel': 'barrelaged',
                  'german|märzen': 'germanic styles'
                  
                 })

In [54]:
def beer_style_mapper(beer_dict):
    for k, v in beer_dict.items():
        beer_df.loc[beer_df.style_name.str.contains(k), "style_name"] = v
    
    return beer_df

In [55]:
beer_style_mapper(beer_dict)

,beer,style_name
0,Beau's All Natural Brewing Company: Beau's Lug...,kolsch
1,Grupo Modelo: Corona,lager
2,Coors Brewing Company: Coors Light,lager
3,Heineken: Heineken Lager,lager
4,Brasserie Dieu De Ciel!: Peche Mortel,imperial st
...,...,...
333,Wellington Brewery: Iron Duke,strong ale
334,Wellington Brewery: Bewitched Belgian,strong ale
335,À la Fût: Hopfenweisse,wheat
336,À la Fût: Cuvée Houblonée I,pale ale


In [56]:
beer_df.shape

(338, 2)

In [57]:
beer_style_names_list = beer_df['style_name'].unique()

In [ ]:
beer_df.to_csv('beer_style_names.csv')
